In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import numpy as np, pandas as pd
from prophet import Prophet
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error

df = pd.read_csv('/kaggle/input/weather-hanoi-2020-2025-normed/merge_weather_energy_hanoi_20202025_norm.csv', parse_dates=["timestamp"])
df

,Temperature,Weather,Precipitation,Chance of snow,Humidity,Wind,Wind Gust,Wind Degree,Wind Direction,Cloud Cover,Visibility,timestamp,is_weekend,season,is_holiday,total_consumption_mw
0,25.4,Patchy rain possible,0.6,0.0,89.0,2.194444,4.388889,295.0,WNW,89.0,9.0,2020-01-01 00:00:00,0,winter,False,1790.10
1,25.1,Partly cloudy,0.0,0.0,90.0,2.611111,5.111111,297.0,WNW,34.0,10.0,2020-01-01 01:00:00,0,winter,False,1452.26
2,24.7,Patchy rain possible,0.0,0.0,91.0,2.805556,5.500000,309.0,NW,87.0,10.0,2020-01-01 02:00:00,0,winter,False,1483.75
3,24.5,Cloudy,0.0,0.0,92.0,2.611111,4.888889,325.0,NW,71.0,10.0,2020-01-01 03:00:00,0,winter,False,1890.07
4,24.1,Patchy rain possible,0.0,0.0,93.0,2.305556,4.000000,326.0,NNW,100.0,10.0,2020-01-01 04:00:00,0,winter,False,1371.23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47444,26.8,Partly cloudy,0.0,0.0,84.0,1.888889,3.388889,109.0,ESE,56.0,10.0,2025-05-30 20:00:00,0,spring,False,2449.28
47445,26.5,Partly cloudy,0.0,0.0,86.0,2.388889,4.194444,126.0,SE,52.0,10.0,2025-05-30 21:00:00,0,spring,False,2554.05
47446,26.1,Patchy rain possible,0.0,0.0,88.0,2.388889,4.194444,149.0,SSE,84.0,10.0,2025-05-30 22:00:00,0,spring,False,1895.41
47447,25.6,Patchy rain possible,0.5,0.0,91.0,2.305556,4.111111,152.0,SSE,76.0,9.0,2025-05-30 23:00:00,0,spring,False,1558.67


In [2]:
CAT_COLS   = ['Weather','Wind Direction','season','is_holiday']
EXOG_NUM   = ['Temperature','Precipitation','Humidity',
              'Wind','Wind Gust','Wind Degree','Cloud Cover','Visibility']
HORIZON    = 24                       # forecast t+1…t+24
TARGET_LAGS   = range(1,49)           # lags of target (1…48)
ROLL_WINDOWS  = [3,6,12,24,48]        # rolling windows
EXOG_LAGS     = [0,1,3,6,12,24]       # lags for exogenous vars
TARGET_COL = 'total_consumption_mw'
TIME_COL   = 'timestamp'

In [3]:
df[TIME_COL] = pd.to_datetime(df[TIME_COL], errors='coerce')
df = df.dropna(subset=[TIME_COL]).sort_values(TIME_COL).set_index(TIME_COL)

In [4]:
df.shape

(47449, 15)

In [5]:
y = df[TARGET_COL].astype(float).asfreq('H').interpolate(limit=3)

/tmp/ipykernel_48/2026607577.py:1: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  y = df[TARGET_COL].astype(float).asfreq('H').interpolate(limit=3)


In [6]:
# ---- split: last 8 weeks test, previous 8 weeks val ----
H8W = 8*7*24
N = len(y)
train_end = N - 2*H8W
valid_end = N - 1*H8W

y_train = y.iloc[:train_end]
y_val   = y.iloc[train_end:valid_end]
y_test  = y.iloc[valid_end:]

print("train/val/test:", len(y_train), len(y_val), len(y_test))
print("ranges:", y_train.index.min(), "→", y_train.index.max(), "|",
                 y_val.index.min(), "→", y_val.index.max(), "|",
                 y_test.index.min(), "→", y_test.index.max())

train/val/test: 44761 1344 1344
ranges: 2020-01-01 00:00:00 → 2025-02-08 00:00:00 | 2025-02-08 01:00:00 → 2025-04-05 00:00:00 | 2025-04-05 01:00:00 → 2025-05-31 00:00:00


In [7]:
def metrics(y_true, y_pred):
    y_true = np.asarray(y_true, float)
    y_pred = np.asarray(y_pred, float)
    mae  = np.mean(np.abs(y_true - y_pred))
    rmse = np.sqrt(np.mean((y_true - y_pred)**2))
    mape = np.nanmean(np.abs((y_true - y_pred)/np.where(y_true==0, np.nan, np.abs(y_true))))*100
    return mae, rmse, mape

def daily_anchors(index):
    # anchor tại 00:00 mỗi ngày có đủ 24h tiếp theo
    start = index.min().normalize()
    end   = (index.max() - pd.Timedelta(hours=23)).normalize()
    if end < start: 
        return pd.DatetimeIndex([])
    return pd.date_range(start, end, freq='D')


In [8]:
from prophet import Prophet

def prophet_day_ahead_recursive(y_full, test_index,
                                cps=0.05, sps=10.0, use_last_days=180):
    rows = []
    anchors = daily_anchors(test_index)
    for a in anchors:
        end_train = a - pd.Timedelta(hours=1)
        y_tr = y_full.loc[:end_train]
        if len(y_tr) < 24*60:
            continue
        if use_last_days is not None:
            y_tr = y_tr.iloc[-24*use_last_days:]

        df_tr = pd.DataFrame({'ds': y_tr.index, 'y': y_tr.values})
        m = Prophet(daily_seasonality=True, weekly_seasonality=True, yearly_seasonality=True,
                    changepoint_prior_scale=cps, seasonality_prior_scale=sps)
        m.fit(df_tr)

        future = pd.DataFrame({'ds': pd.date_range(a, a+pd.Timedelta(hours=23), freq='H')})
        yhat = m.predict(future)[['ds','yhat']]
        y_true = y_full.loc[a:a+pd.Timedelta(hours=23)]

        for i, row in yhat.iterrows():
            ts = row['ds']
            if ts in y_true.index:
                h = (i % 24) + 1  # 1..24
                rows.append({"model":"Prophet (no exog)",
                             "ds": ts, "h": h,
                             "y_true": float(y_true.loc[ts]),
                             "y_pred": float(row['yhat'])})
    return pd.DataFrame(rows)

df_prophet = prophet_day_ahead_recursive(y, y_test.index, cps=0.05, sps=10.0, use_last_days=180)

09:38:00 - cmdstanpy - INFO - Chain [1] start processing
09:38:00 - cmdstanpy - INFO - Chain [1] done processing
/tmp/ipykernel_48/701258844.py:20: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  future = pd.DataFrame({'ds': pd.date_range(a, a+pd.Timedelta(hours=23), freq='H')})
09:38:00 - cmdstanpy - INFO - Chain [1] start processing
09:38:01 - cmdstanpy - INFO - Chain [1] done processing
/tmp/ipykernel_48/701258844.py:20: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  future = pd.DataFrame({'ds': pd.date_range(a, a+pd.Timedelta(hours=23), freq='H')})
09:38:01 - cmdstanpy - INFO - Chain [1] start processing
09:38:01 - cmdstanpy - INFO - Chain [1] done processing
/tmp/ipykernel_48/701258844.py:20: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  future = pd.DataFrame({'ds': pd.date_range(a, a+pd.Timedelta(hours=23), freq='H')})
09:3

In [9]:
def mae(y, p): y, p = np.asarray(y), np.asarray(p); return float(np.mean(np.abs(y-p)))
def rmse(y, p): y, p = np.asarray(y), np.asarray(p); return float(np.sqrt(np.mean((y-p)**2)))
def mape(y, p):
    y, p = np.asarray(y, float), np.asarray(p, float)
    denom = np.where(y==0, np.nan, np.abs(y))
    return float(np.nanmean(np.abs((y-p)/denom))*100)

def summarize(df_err, model_name):
    # overall
    m_mae  = mae(df_err['y_true'], df_err['y_pred'])
    m_rmse = rmse(df_err['y_true'], df_err['y_pred'])
    m_mape = mape(df_err['y_true'], df_err['y_pred'])
    print(f"\nDay-ahead (recursive) OVERALL on TEST — {model_name}:")
    print(f"  MAE  = {m_mae:,.3f}")
    print(f"  RMSE = {m_rmse:,.3f}")
    print(f"  MAPE = {m_mape:,.3f}%")

    # per-horizon
    print("\n  Per-horizon RMSE/MAPE (h=1..24):")
    print("   h       RMSE       MAPE_%")
    per = (df_err.groupby('h')
                .apply(lambda g: pd.Series({
                    'RMSE': rmse(g['y_true'], g['y_pred']),
                    'MAPE_%': mape(g['y_true'], g['y_pred'])
                }))
                .reset_index())
    for _, r in per.sort_values('h').iterrows():
        print(f"{int(r['h']):2d} {r['RMSE']:10.6f} {r['MAPE_%']:10.6f}")
    return per

In [10]:
per_prophet = summarize(df_prophet, "Prophet (no weather)")


Day-ahead (recursive) OVERALL on TEST — Prophet (no weather):
  MAE  = 597.146
  RMSE = 739.699
  MAPE = 25.232%

  Per-horizon RMSE/MAPE (h=1..24):
   h       RMSE       MAPE_%
 1 718.716185  50.628474
 2 558.749329  39.600143
 3 476.124123  32.828622
 4 475.701764  32.736264
 5 525.329570  35.999651
 6 508.074312  31.169805
 7 325.624802  14.919822
 8 343.915641  10.297129
 9 439.396811  12.928112
10 767.136689  18.092885
11 1062.603301  23.518094
12 1213.403427  27.045599
13 1264.867606  26.296708
14 1160.338043  24.619275
15 959.949069  21.456346
16 871.870851  21.392372
17 613.642465  17.823519
18 460.259697  10.243838
19 431.214993  14.616459
20 569.152841  18.597368
21 648.688909  21.231137
22 591.598230  19.893037
23 809.008575  37.608860
24 746.340149  42.018046


/tmp/ipykernel_48/4235729826.py:22: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: pd.Series({
